In [ ]:
!pip -q install groq

In [ ]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

## basic example

llama3-groq-70b-8192-tool-use-preview
llama3-groq-8b-8192-tool-use-preview

In [ ]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-groq-8b-8192-tool-use-preview",
)

print(chat_completion.choices[0].message.content)

Fast language models are essential for various applications in natural language processing (NLP) due to their ability to process and generate text quickly. Here are some reasons why they are important:

1. **Real-time Processing**: Fast language models enable real-time processing of text, which is crucial for applications that require immediate responses, such as chatbots, virtual assistants, and live transcription services.
2. **Scalability**: They can handle large volumes of text efficiently, making them ideal for big data applications and tasks that involve processing massive amounts of text.
3. **Speed of Development**: Fast language models accelerate the development of NLP applications by providing a foundation for rapid prototyping and testing.
4. **Improved User Experience**: By responding quickly, fast language models enhance the user experience in applications like customer service, where fast resolution is key.
5. **Enhanced Analytics**: They facilitate the analysis of large 

# **Tool Use with Groq**

In [ ]:
from groq import Groq
import json

client = Groq()
MODEL = 'llama3-groq-70b-8192-tool-use-preview'

def calculate(expression):
    """Evaluate a mathematical expression"""
    try:
        result = eval(expression)
        print(expression)
        return json.dumps({"result": result})
    except:
        return json.dumps({"error": "Invalid expression"})

def run_conversation(user_prompt):
    messages=[
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Evaluate a mathematical expression",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate",
                        }
                    },
                    "required": ["expression"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "calculate": calculate,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            print(tool_call)
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                expression=function_args.get("expression")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        return second_response.choices[0].message.content

user_prompt = "What is 25 x 4 + 10 - 9.11?"
print(run_conversation(user_prompt))

ChatCompletionMessageToolCall(id='call_7d4b', function=Function(arguments='{"expression": "25*4+10-9.11"}', name='calculate'), type='function')
25*4+10-9.11
The result of 25 x 4 + 10 - 9.11 is 100.89.


In [ ]:
user_prompt = "What is 6 squared?"
print(run_conversation(user_prompt))

ChatCompletionMessageToolCall(id='call_6nys', function=Function(arguments='{"expression": "6^2"}', name='calculate'), type='function')
6^2
6 squared is 36.


In [ ]:
user_prompt = "What is 25 times 4 plus ten?"
print(run_conversation(user_prompt))

ChatCompletionMessageToolCall(id='call_2x9a', function=Function(arguments='{"expression": "25*4+10"}', name='calculate'), type='function')
25*4+10
The result of 25 times 4 plus 10 is 110.


In [ ]:
user_prompt = "How are you today?"
print(run_conversation(user_prompt))

None


In [ ]:
!pip -q install wikipedia duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 40.2 MB/s eta 0:00:00


In [ ]:
from groq import Groq
import json
import requests
import wikipedia
from duckduckgo_search import DDGS

client = Groq()
MODEL = 'llama3-groq-70b-8192-tool-use-preview'

def duckduckgo_search(query):
    """Perform a DuckDuckGo search using the duckduckgo_search library"""
    print(f"Using DuckDuckGo for {query}")

    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=3))

        if not results:
            return f"No results found for '{query}'. The search may have been too specific or there might be no relevant information available."

        formatted_results = "\n".join([f"- {result['title']}: {result['body']}" for result in results])
        return f"Here are some search results for '{query}':\n{formatted_results}"

    except Exception as e:
        return f"An error occurred while searching for '{query}': {str(e)}"

def wikipedia_search(query):
    """Perform a Wikipedia search"""
    try:
        print(f"Using Wikipedia for {query}")
        page = wikipedia.page(query)
        summary = wikipedia.summary(query, sentences=2)
        return json.dumps({"title": page.title, "summary": summary, "url": page.url})
    except:
        return json.dumps({"error": "No Wikipedia page found for the query"})

def run_conversation(user_prompt):
    messages=[
        {
            "role": "system",
            "content": "You are a search assistant. Use the duckduckgo_search function to perform web \
            searches and the wikipedia_search function to find information on Wikipedia. if you don't \
            need either of these then just reply normally"
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "duckduckgo_search",
                "description": "Perform a DuckDuckGo search",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query",
                        }
                    },
                    "required": ["query"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "wikipedia_search",
                "description": "Perform a Wikipedia search",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query",
                        }
                    },
                    "required": ["query"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "duckduckgo_search": duckduckgo_search,
            "wikipedia_search": wikipedia_search,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            print(tool_call)
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                query=function_args.get("query")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        return second_response.choices[0].message.content

user_prompt = "What is the capital of France and what's a famous landmark there?"
print(run_conversation(user_prompt))

ChatCompletionMessageToolCall(id='call_2j7f', function=Function(arguments='{"query": "capital of France"}', name='wikipedia_search'), type='function')
Using Wikipedia for capital of France
ChatCompletionMessageToolCall(id='call_k96v', function=Function(arguments='{"query": "famous landmark in France"}', name='wikipedia_search'), type='function')
Using Wikipedia for famous landmark in France
The capital of France is Paris. A famous landmark there is the Eiffel Tower, which is an iconic symbol of France and one of the most recognizable structures in the world.


In [ ]:
user_prompt = "What are the latest news headlines about artificial intelligence?"
print(run_conversation(user_prompt))

ChatCompletionMessageToolCall(id='call_884h', function=Function(arguments='{"query": "latest news on artificial intelligence"}', name='duckduckgo_search'), type='function')
Using DuckDuckGo for latest news on artificial intelligence
Here are the latest news headlines on artificial intelligence:
1. "AI News | Latest Headlines and Developments | Reuters" - July 11, 2024, covering how customers have booked out South Korean cosmetics giant AmorePacific's new artificial intelligence beauty lab.
2. "Exclusive: OpenAI working on new reasoning technology under code name 'Strawberry'." - OpenAI is working on a novel approach to its artificial intelligence models.
3. "AI News & Artificial Intelligence | TechCrunch" - TechCrunch covers the latest news and trends on artificial intelligence and machine learning tech, including generative AI, speech recognition, and predictive analytics.


In [ ]:
user_prompt = "Tell me about the company Groq?"
print(run_conversation(user_prompt))

None


In [ ]:
user_prompt = "How are you today?"
print(run_conversation(user_prompt))

None


In [ ]:
import json
import requests
import wikipedia
from groq import Groq

client = Groq()
MODEL = 'llama3-groq-70b-8192-tool-use-preview'

def duckduckgo_search(query):
    """Perform a DuckDuckGo search using the duckduckgo_search library"""
    print(f"Using DuckDuckGo for {query}")

    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=3))

        if not results:
            return f"No results found for '{query}'. The search may have been too specific or there might be no relevant information available."

        formatted_results = "\n".join([f"- {result['title']}: {result['body']}" for result in results])
        return f"Here are some search results for '{query}':\n{formatted_results}"

    except Exception as e:
        return f"An error occurred while searching for '{query}': {str(e)}"

def wikipedia_search(query):
    """Perform a Wikipedia search"""
    try:
        print(f"Using Wikipedia for {query}")
        page = wikipedia.page(query)
        summary = wikipedia.summary(query, sentences=2)
        return f"Wikipedia article: {page.title}\nSummary: {summary}\nURL: {page.url}"
    except:
        return f"No Wikipedia page found for the query '{query}'"

def run_conversation(user_prompt):
    messages=[
        {
            "role": "system",
            "content": "You are a search assistant. Use the duckduckgo_search function to perform web \
            searches and the wikipedia_search function to find information on Wikipedia. If you don't \
            need either of these then just reply normally. When using search results, summarize and \
            integrate the information into your response."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "duckduckgo_search",
                "description": "Perform a DuckDuckGo search",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query",
                        }
                    },
                    "required": ["query"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "wikipedia_search",
                "description": "Perform a Wikipedia search",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query",
                        }
                    },
                    "required": ["query"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "duckduckgo_search": duckduckgo_search,
            "wikipedia_search": wikipedia_search,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                query=function_args.get("query")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        return second_response.choices[0].message.content
    else:
        return response_message.content

# Example usage
user_prompt = "What are the latest news headlines about artificial intelligence?"
print(run_conversation(user_prompt))


Using DuckDuckGo for latest news on artificial intelligence
Here are the latest news headlines on artificial intelligence:
1. OpenAI is working on a new project called "Strawberry," which involves a novel approach to AI technology.
2. In South Korea, a new AI beauty lab has been fully booked, where robots custom mix face products using the latest technology.
3. OpenAI has introduced a set of levels to track progress towards building superintelligent AI, a significant step in monitoring AI advancements.


In [ ]:
user_prompt = "How are you today?"
print(run_conversation(user_prompt))

I'm doing well, thank you for asking. How can I assist you today?


In [ ]:
user_prompt = "please search and tell me about the company Groq.com?"
print(run_conversation(user_prompt))

Using Wikipedia for Groq.com
Using DuckDuckGo for Groq.com
Based on the information I found, Groq, Inc. is an American artificial intelligence (AI) company that builds an AI accelerator application-specific integrated circuit (ASIC) called the Language Processing Unit (LPU) and related hardware to accelerate the inference performance of AI workloads. They provide cloud and on-premises solutions at scale for AI applications, with a headquarters in Silicon Valley and founded in 2016. The LPU and related systems are designed, fabricated, and assembled in North America. Additionally, GroqCloud provides the world's fastest AI inference for large language models (LLMs) via its Groq LPU Inference Engine. You can access the GroqCloud API for enterprise, public sector, and developer projects, and compare prices and performance with other providers.


In [ ]:
user_prompt = "What are the latest news headlines about artificial intelligence?"
print(run_conversation(user_prompt))

Using DuckDuckGo for latest news on artificial intelligence
Here are the latest news headlines on artificial intelligence:
1. "AI News | Latest Headlines and Developments | Reuters": This article discusses how customers have booked out South Korean cosmetics giant AmorePacific's new artificial intelligence beauty lab, where robots custom mix face products and the latest technology recommends the most suitable skincare routines.
2. "Exclusive: OpenAI working on new reasoning technology under code name 'Strawberry'": OpenAI, the maker of ChatGPT, is working on a novel approach to its artificial intelligence models in a project code-named 'Strawberry', according to a person familiar with the matter and internal documents.
3. "OpenAI Sets Levels to Track Progress Toward Superintelligent AI - Bloomberg": OpenAI has come up with a set of five levels to track its progress toward building artificial intelligence software capable of outperforming humans, the startup's latest effort to help peop

In [ ]:
import json
import requests
import wikipedia
from groq import Groq
from duckduckgo_search import DDGS

client = Groq()
MODEL = 'llama3-groq-70b-8192-tool-use-preview'



def duckduckgo_search(query):
    """Perform a DuckDuckGo search using the duckduckgo_search library"""
    print(f"Using DuckDuckGo for {query}")

    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=3))

        if not results:
            return f"No results found for '{query}'. The search may have been too specific or there might be no relevant information available."

        formatted_results = "\n".join([f"- {result['title']}: {result['body']}" for result in results])
        return f"Here are some search results for '{query}':\n{formatted_results}"

    except Exception as e:
        return f"An error occurred while searching for '{query}': {str(e)}"

def wikipedia_search(query):
    """Perform a Wikipedia search"""
    try:
        print(f"Using Wikipedia for {query}")
        page = wikipedia.page(query)
        summary = wikipedia.summary(query, sentences=2)
        return f"Wikipedia article: {page.title}\nSummary: {summary}\nURL: {page.url}"
    except:
        return f"No Wikipedia page found for the query '{query}'"


def run_conversation(user_prompt):
    messages = [
        {
            "role": "system",
            "content": "You are a search assistant. Use the internet_search function to perform web \
            searches and the wikipedia_search function to find information on Wikipedia. You can use \
            multiple tools if needed. If you don't need either of these then just reply normally. \
            When using search results, summarize and integrate the information into your response."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "internet_search",
                "description": "use this to search about a topic on the internet",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query",
                        }
                    },
                    "required": ["query"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "wikipedia_search",
                "description": "Perform a Wikipedia search",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query",
                        }
                    },
                    "required": ["query"],
                },
            },
        }
    ]

    available_functions = {
        "internet_search": duckduckgo_search,
        "wikipedia_search": wikipedia_search,
    }

    tool_outputs = []
    max_iterations = 3  # Limit the number of tool calls to prevent infinite loops

    for _ in range(max_iterations):
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            max_tokens=4096
        )

        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls

        if not tool_calls:
            # If no more tool calls are needed, return the final response
            return {
                "final_response": response_message.content,
                "tool_outputs": tool_outputs
            }

        messages.append(response_message)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                query=function_args.get("query")
            )
            tool_outputs.append({
                "tool": function_name,
                "query": function_args.get("query"),
                "result": function_response
            })
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

    # If we've reached the maximum number of iterations, return the last response
    return {
        "final_response": "Maximum number of tool calls reached. Here's the last response:\n" + response_message.content,
        "tool_outputs": tool_outputs
    }

# Example usage
user_prompt = "Compare the latest AI developments in healthcare and finance. Include specific examples from recent news."
result = run_conversation(user_prompt)
print("Final Response:", result["final_response"])
print("\nTool Outputs:")
for output in result["tool_outputs"]:
    print(f"\nTool: {output['tool']}")
    print(f"Query: {output['query']}")
    print(f"Result: {output['result']}")

Using DuckDuckGo for latest AI developments in healthcare
Using DuckDuckGo for latest AI developments in finance
Final Response: Based on the latest news, AI is transforming both healthcare and finance by improving outcomes, patient safety, and the accessibility of high-quality care in healthcare, and by potentially reshaping the finance industry, making it more efficient and accessible. For instance, in healthcare, AI is being used to improve diagnostics, patient care, and health management. In finance, AI is enhancing fraud detection, personalizing customer service, and optimizing financial models.

Tool Outputs:

Tool: internet_search
Query: latest AI developments in healthcare
Result: Here are some search results for 'latest AI developments in healthcare':
- The potential for artificial intelligence to transform healthcare ...: Artificial intelligence (AI) has the potential to transform care delivery by improving health outcomes, patient safety, and the affordability and accessibil

In [ ]:
# Example usage
user_prompt = "search when do the olympics start?"
result = run_conversation(user_prompt)
print("Final Response:", result["final_response"])
print("\nTool Outputs:")
for output in result["tool_outputs"]:
    print(f"\nTool: {output['tool']}")
    print(f"Query: {output['query']}")
    print(f"Result: {output['result']}")

Using DuckDuckGo for When do the Olympics start
Final Response: The 2024 Paris Olympics are scheduled to start on July 26, 2024, with the opening ceremony along the Seine River. The first events, rugby and soccer, will take place on July 24, 2024. The games will end on August 11, 2024.

Tool Outputs:

Tool: internet_search
Query: When do the Olympics start
Result: Here are some search results for 'When do the Olympics start':
- Paris 2024 - Olympics.com: Please note the competition schedule is subject to change until the conclusion of the Olympic Games. The surfing competition will be held over 4 days within a period of 9 days, depending on the weather conditions. Timing and Scoring provided by Omega / Official Results powered by Atos. Official Paris 2024 Olympic schedule.
- Paris 2024 Olympic Games full schedule and day-by-day competitions: Find out when and where your favourite sports and events are happening at the Olympic Games Paris 2024. The Games start on 26 July and end on 11 A